# Day 1 - Lab 2: Generating a Product Requirements Document (PRD)

**Objective:** Use the structured `day1_user_stories.json` artifact from the previous lab to generate a formal, comprehensive Product Requirements Document (PRD) in markdown format.

**Estimated Time:** 60 minutes

**Introduction:**
With a validated set of user stories, we can now create a higher-level planning document: the PRD. A PRD serves as the source of truth for the product team, outlining the project's purpose, features, and requirements. In this lab, you will use an LLM to synthesize the detailed user stories into this formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM) and load the JSON artifact from the previous lab.

**Model Selection:**
You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in `utils.py`, such as `"gemini-2.5-flash"` or `"meta-llama/Llama-3.3-70B-Instruct"`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the user stories JSON file and the PRD template.
- `save_artifact()`: To save our generated PRD and Pydantic model.

In [4]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the artifact from Lab 1
artifacts_path = os.getenv("ARTIFACTS_PATH", "../../artifacts")
user_stories_str = load_artifact("artifacts/day1_user_stories.json")
if user_stories_str:
    user_stories_data = json.loads(user_stories_str)
else:
    user_stories_data = []

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a Simple PRD

**Task:** Use the loaded user stories to generate a simple PRD.

**Instructions:**
1. Create a prompt that instructs the LLM to act as a Product Manager.
2. Provide the `user_stories_data` as context.
3. Ask the LLM to generate a PRD with three sections: "Introduction", "User Personas", and "Features / User Stories".

**Expected Quality:** A clean markdown document that correctly summarizes the provided user stories into the requested sections.

In [5]:
# TODO: Write a prompt to generate a simple PRD.
simple_prd_prompt = f"""
You are a Product Manager tasked with creating a Product Requirements Document (PRD) for a new hire onboarding tool.

Use the following JSON data containing user stories as your primary source of information:
<user_stories>
{user_stories_str}
</user_stories>

Your task:
- Write a PRD in markdown format.
- Ensure the document is clear, concise, and professional.

The PRD must include the following sections:

1. **Introduction:**
   - Provide a brief overview of the project's purpose, goals, and the problem it aims to solve.
   - Highlight the importance of the onboarding tool for the organization.

2. **User Personas:**
   - Summarize the key user personas derived from the user stories.
   - Include their roles, goals, and how they interact with the onboarding tool.

3. **Features / User Stories:**
   - List the features and user stories from the `user_stories_data` variable.
   - For each user story, include:
     - The persona it addresses.
     - The feature it relates to.
     - The user story in the format: "As a [persona], I want to [do something], so that [benefit]."
     - The acceptance criteria in Gherkin format (Given/When/Then).

Output the PRD in markdown format, ensuring proper section headers and formatting.
"""

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

--- Generating Simple PRD ---
```markdown
# Product Requirements Document (PRD)

## Introduction
The New Hire Onboarding Tool is designed to streamline and enhance the onboarding process for new employees, HR managers, and department managers. This tool aims to solve the common challenges of onboarding by providing a centralized platform that offers personalized learning paths, progress tracking, compliance training, and various other features essential for an effective onboarding experience. The onboarding tool is crucial for the organization as it facilitates a smooth transition for new hires, ensures compliance with company policies, and fosters an engaging and supportive work environment.

## User Personas

### New Employee
- **Role:** Newly hired staff member.
- **Goals:** Acclimate quickly to the company, complete onboarding tasks efficiently, and understand their role and the company's culture.
- **Interaction:** Uses the tool to access learning paths, track progress, complete c

### Challenge 2 (Intermediate): Generating a PRD from a Template

**Task:** Instead of just listing sections, we will now provide the LLM with a formal template to ensure the PRD's structure is consistent and complete.

**Instructions:**
1. First, load the contents of `templates/prd_template.md` into a variable.
2. Create a new prompt that instructs the LLM to act as a Senior Product Manager.
3. Provide both the `user_stories_data` and the `prd_template_content` as context.
4. Instruct the LLM to populate the template with the information from the user stories, ensuring every section of the template is filled out.

> **Tip:** The template has sections like 'Success Metrics' and 'Out of Scope' that aren't in the user stories. This is your chance to guide the LLM's creativity! Instruct it to infer logical content for these sections based on the project's overall goal.

**Expected Quality:** A complete PRD that strictly follows the structure of the provided template file, demonstrating the LLM's ability to perform structured content generation.

In [7]:
# Load the PRD template with enhanced error handling and user-friendly messages
try:
    prd_template_content = load_artifact("templates/prd_template.md")
    if not prd_template_content:
        raise FileNotFoundError("The PRD template file is empty. Please check the file at 'templates/prd_template.md'.")
    print(f"✅ Successfully loaded PRD template ({len(prd_template_content)} characters).")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("   Ensure the file exists and is not empty.")
    prd_template_content = None
except Exception as e:
    print(f"❌ An unexpected error occurred while loading the PRD template: {e}")
    print("   Please check the file path and permissions.")
    prd_template_content = None

# TODO: Write a prompt to populate the PRD template.
template_prd_prompt = f"""
You are a Senior Product Manager tasked with creating a comprehensive and formal Product Requirements Document (PRD) for a new hire onboarding tool.

Your responsibilities include:
1. Using the provided PRD template as the structure for the document.
2. Incorporating the user stories JSON data to populate relevant sections of the template.
3. Ensuring that every section of the template is completed, including sections like 'Success Metrics' and 'Out of Scope', which may not be explicitly covered in the user stories. For these sections, infer logical and reasonable content based on:
   - The overall project goal of creating a new hire onboarding tool.
   - The roles and goals of the personas described in the user stories.

Context:
<prd_template>
{prd_template_content}
</prd_template>

<user_stories_json>
{user_stories_str}
</user_stories_json>

Output:
Generate the completed PRD in markdown format. Ensure the document:
- Strictly follows the structure of the provided template.
- Is clear, concise, and professional.
- Demonstrates the ability to synthesize structured content from the provided data.
"""

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

# Diagnostic: Check template file existence and path resolution
print("=== TEMPLATE DIAGNOSTIC ===")
print("Current working directory:", os.getcwd())

# Check if templates directory exists
templates_dir = os.path.join(project_root, "templates")
print(f"Project root: {project_root}")
print(f"Templates directory: {templates_dir}")
print(f"Templates directory exists: {os.path.exists(templates_dir)}")

if os.path.exists(templates_dir):
    print("Contents of templates directory:")
    for item in os.listdir(templates_dir):
        full_path = os.path.join(templates_dir, item)
        print(f"  - {item} ({'file' if os.path.isfile(full_path) else 'directory'})")

# Check the specific template file
template_file = os.path.join(project_root, "templates", "prd_template.md")
print(f"\nPRD template file path: {template_file}")
print(f"PRD template file exists: {os.path.exists(template_file)}")

if os.path.exists(template_file):
    print(f"File size: {os.path.getsize(template_file)} bytes")
else:
    print("❌ Template file not found!")
    
print("==========================")

✅ Successfully loaded PRD template (4217 characters).
--- Generating PRD from Template ---
```markdown
# Product Requirements Document: New Hire Onboarding Tool

| Status | **Draft** |
| :--- | :--- |
| **Author** | Senior Product Manager |
| **Version** | 1.0 |
| **Last Updated** | [Date] |

## 1. Executive Summary & Vision
The New Hire Onboarding Tool is designed to streamline and enhance the onboarding experience for new employees, HR managers, and department managers. By providing a centralized platform with personalized learning paths, progress tracking, compliance modules, and integration with existing HR systems, the tool aims to improve new hire efficiency and engagement while reducing the administrative burden on HR and department managers. The ultimate vision is to create a seamless onboarding experience that accelerates new hire productivity and fosters a sense of belonging within the organization.

## 2. The Problem

**2.1. Problem Statement:**
New hires currently face a fr

### Challenge 3 (Advanced): Programmatic Validation with Pydantic

**Task:** We will now create a Pydantic model to represent the structure of our PRD. This allows us to programmatically validate any PRD, ensuring it meets our standards before it's accepted as a formal artifact.

**Instructions:**
1.  Prompt the LLM to generate a Pydantic model that reflects the structure of the `prd_template.md`. The model should have fields for each major section (e.g., `introduction: str`, `user_personas: List[str]`, `user_stories: List[Dict]`).
2.  Save this generated model code to a file named `app/validation_models/prd_model.py`.
3.  While we won't write the full validation script in this lab, generating the Pydantic model itself is the key advanced step. It creates a reusable, code-based standard for our documentation.

**Expected Quality:** A Python file containing a valid Pydantic model that can be used in the future to validate PRD documents automatically. This represents a shift from manual document review to automated governance.

In [ ]:
# TODO: Write a prompt to generate a Pydantic model for the PRD.
# Tip: Be specific. Tell the LLM to create a class named 'ProductRequirementsDocument' and to use appropriate types from Python's 'typing' library.
pydantic_model_prompt = f"""
# Your prompt here
"""

print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Clean up the code if it's wrapped in markdown fences
    if '```' in pydantic_model_code:
        pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()
    
    print("\n--- Generated Pydantic Model ---")
    print(pydantic_model_code)

    # Save the generated Pydantic model code to a file.
    model_path = "app/validation_models/prd_model.py"
    save_artifact(pydantic_model_code, model_path)
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge
if prd_from_template_output:
    save_artifact(prd_from_template_output, "artifacts/day1_prd.md")

## Lab Conclusion

Excellent work! You have now taken the structured user stories from the first lab and synthesized them into a formal Product Requirements Document. You also created a Pydantic model to enforce the structure of this document, introducing automated governance into your workflow. The `day1_prd.md` artifact will be the primary input for Day 2, where we will begin designing our system's architecture and database.

> **Key Takeaway:** Using an LLM to populate a pre-defined template is a powerful pattern for creating consistent, high-quality documentation at scale. It combines the LLM's language skills with your required structure.